## Mixed Model NBA Shooting Predictor

#### Table of contents
    1. Motivation
    2. Exploratory Data Analysis

#### Motivation

The goal of this project to is to explore mixed effect modeling on a players shooting ability and build a model that can accurately predict a players ability to make a shot given different features. Some features that will be explored include where the player is located on the floor and the opposing teams defender/team defensive ratings. 
    
With mixed effects models it is important to first determine whether the variables should be treated as random effects or fixed effects. If the effect will be similar amoung groups (players) the effect will be fixed, if they effects are different among groups the effects will be randon



In [1]:
#import libraries
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
from py_ball import player, playbyplay

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
HEADERS = {'Connection': 'keep-alive',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'Referer': 'stats.nba.com',
           'x-nba-stats-origin': 'stats',
           'x-nba-stats-token': 'true',
           'Accept-Language': 'en-US,en;q=0.9',
           "X-NewRelic-ID": "VQECWF5UChAHUlNTBwgBVw==",
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6)' +\
                         ' AppleWebKit/537.36 (KHTML, like Gecko)' + \
                         ' Chrome/81.0.4044.129 Safari/537.36'}

#### Exploratory 

Exploring data available through the py_ball api. First creating a baseline model using just the players location on the floor and the type of shot they are taking.

In [ ]:
#getting the shotchart for the 22-23 regular season
league_id = '00' #NBA
player_id = '0' #All players

season = '2022-23'
x=season.split('0',1)[1]
season_split = x.split('-',1)[0]
all_df = pd.DataFrame({})

for x in range(1,1231):
    print('Game #' + str(x))
    t0 = time.time()
    game_id = '002'+ season_split +'0'+ str(x).zfill(4)
    shots = player.Player(headers=HEADERS,
                      endpoint='shotchartdetail',
                      league_id=league_id,
                      player_id=player_id,
                      game_id=game_id,
                      season=season)
    shot_df = pd.DataFrame(shots.data['Shot_Chart_Detail'])
    all_df = pd.concat([all_df, shot_df], axis=0).reset_index(drop=True)
    delay = time.time() - t0
    print('Waiting ' + str(10*delay) + 's')
    time.sleep(delay)
    
all_df.to_csv('shotchart.csv', index=False)

In [5]:
pd.set_option('display.max_columns', None)
shot_chart = pd.read_csv(r'C:\Users\nmani\OneDrive\Desktop\shotchart.csv')


In [6]:
shot_chart

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,22200001,7,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,11,38,Missed Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,12,-118,50,1,0,20221018,BOS,PHI
1,Shot Chart Detail,22200001,11,203935,Marcus Smart,1610612738,Boston Celtics,1,11,15,Made Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,13,120,55,1,1,20221018,BOS,PHI
2,Shot Chart Detail,22200001,12,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,5,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,50,135,1,0,20221018,BOS,PHI
3,Shot Chart Detail,22200001,14,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,3,Made Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,1,1,20221018,BOS,PHI
4,Shot Chart Detail,22200001,15,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,46,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-232,49,1,1,20221018,BOS,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217215,Shot Chart Detail,22201230,634,1630541,Moses Moody,1610612744,Golden State Warriors,4,1,11,Made Shot,Cutting Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,11,3,1,1,20230409,POR,GSW
217216,Shot Chart Detail,22201230,636,1631219,John Butler Jr.,1610612757,Portland Trail Blazers,4,1,0,Made Shot,Jump Shot,3PT Field Goal,Right Corner 3,Right Side(R),24+ ft.,23,224,84,1,1,20230409,POR,GSW
217217,Shot Chart Detail,22201230,638,1630228,Jonathan Kuminga,1610612744,Golden State Warriors,4,0,44,Missed Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,23,199,126,1,0,20230409,POR,GSW
217218,Shot Chart Detail,22201230,640,1631116,Patrick Baldwin,1610612744,Golden State Warriors,4,0,36,Missed Shot,Driving Floating Bank Jump Shot,2PT Field Goal,In The Paint (Non-RA),Left Side(L),8-16 ft.,8,-71,52,1,0,20230409,POR,GSW


In [9]:
shot_chart['ACTION_TYPE'].unique()

array(['Turnaround Fadeaway shot', 'Driving Floating Bank Jump Shot',
       'Driving Floating Jump Shot', 'Tip Layup Shot', 'Jump Shot',
       'Driving Layup Shot', 'Fadeaway Jump Shot', 'Running Layup Shot',
       'Cutting Layup Shot', 'Step Back Jump shot',
       'Driving Reverse Layup Shot', 'Running Jump Shot', 'Hook Shot',
       'Running Dunk Shot', 'Layup Shot',
       'Driving Finger Roll Layup Shot', 'Pullup Jump shot',
       'Running Reverse Layup Shot', 'Turnaround Bank Hook Shot',
       'Cutting Dunk Shot', 'Driving Hook Shot', 'Hook Bank Shot',
       'Running Finger Roll Layup Shot', 'Floating Jump shot',
       'Turnaround Hook Shot', 'Driving Dunk Shot',
       'Alley Oop Layup shot', 'Running Pull-Up Jump Shot',
       'Putback Layup Shot', 'Putback Dunk Shot',
       'Turnaround Fadeaway Bank Jump Shot', 'Reverse Layup Shot',
       'Running Alley Oop Dunk Shot', 'Tip Dunk Shot', 'Dunk Shot',
       'Alley Oop Dunk Shot', 'Jump Bank Shot', 'Turnaround Jump Shot'

In [ ]:
Angle and Location, 